In [ ]:
from IPython.core.display import HTML
with open('style.css') as file:
    css = file.read()
HTML(css)

In [ ]:
# Autload python modules by default
%load_ext autoreload
%autoreload 2

# Convert notebooks to python, so they can be loaded effiently
from utils.jupyter_loader import JupyterLoader

loader = JupyterLoader()
loader.load_all()

# Iterative Deepening

The algorithms shown so far have the advantage of a low memory consumption, as usual in a depth-first searches. Thus, at most the current path must always be kept in memory. A disadvantage is, however, that it is difficult to build in a time break. If the search is simply aborted after a certain time, some paths have been evaluated completely, while other paths have not been considered at all. Another problem currently with the `AlphaBeta Pruning Algorithm` is that in the optimal case, good moves should be considered first, in order to be able to prune as many paths as possible. So far, however, there is no heuristic to perform this sorting of the moves.

The idea of iterative depth-first search provides a possible solution to both problems. In contrast to the normal depth-first search, the depth is iteratively increased until the maximum depth is reached. The obvious disadvantage of this is more overhead, since nodes that have already been analyzed are looked at again in the next iterative pass. However, as seen in previous examples, the number of new nodes increases so dramatically for each increase in search depth that re-analyzing previous nodes is negligible.

The advantage of iterative depth-first search, however, is that the results obtained from previous runs with lower search depths can be used. On the one hand, these results can be returned if the time limit expires in the next run. On the other hand, based on the previous run, the sorting of moves for the next one can be done. 

At first only the iterative deepening algorithmn itself shall be implemented in a new class `IterativeAlphaBeta`, which inherits from `AlphaBetaEngine`.

In [ ]:
from converted_notebooks.s10_alpha_beta_engine import AlphaBetaEngine
from converted_notebooks.s08_evaluation import Evaluator


class IterativeAlphaBeta(AlphaBetaEngine):

    def __init__(self, evaluator: Evaluator, max_look_ahead_depth: int):
        self.evaluator = evaluator
        self.max_look_ahead_depth = max_look_ahead_depth

The `_evaluate_move` method must be adapted so that the depth is now a parameter.

In [ ]:
import chess
from converted_notebooks.s04_engine_interface import ScoredMove, LowestScore, HighestScore


def _evaluate_move(self, board: chess.Board, move: chess.Move, depth: int):
    board.push(move)
    score = self._value(board, depth - 1, LowestScore, HighestScore)
    board.pop()
    return ScoredMove(score=score.white(), move=move)


IterativeAlphaBeta._evaluate_move = _evaluate_move

In the `_evaluate_moves` method, a simple loop can now be built in, which increases the depth step by step until the desired maximum depth is reached.

In [ ]:
import logging


def _evaluate_moves(self, board: chess.Board):
    logging.info(f"Max depth: {self.max_look_ahead_depth}")

    for depth in range(1, self.max_look_ahead_depth + 1):
        scoredMoves = [
            self._evaluate_move(board, move, depth)
            for move in board.legal_moves
        ]

        logging.info(f"Depth {depth}")
        logging.debug(f"result: {scoredMoves}\n")

    return scoredMoves


IterativeAlphaBeta._evaluate_moves = _evaluate_moves

Again, it can be verified that the new implementation still evaluates the already known position in the same way.

In [ ]:
import random
from converted_notebooks.s09_minimax_engine import middlegame_board, result_minimax
from converted_notebooks.s08_evaluation import standard_evaluator

random.seed(42)

engine = IterativeAlphaBeta(
    evaluator=standard_evaluator, max_look_ahead_depth=3
)
result_iterativeAlphaBeta = engine.analyse(middlegame_board)

assert result_iterativeAlphaBeta == result_minimax

# Transposition tables

In the last section of this chapter we will look at transposition tables. Essentially, they are a cache for the search tree that serves several purposes. The first use case and eponym are transpositions. In chess one speaks of a transposition, if the same position is obtained by different move sequences. In this case, with the help of a cache, the position has to be evaluated only once. In some algorithms, such as the ['MTD(f) algorithm'](https://www.chessprogramming.org/MTD(f)), a node of the search tree is also evaluated several times in succession. In this case it is therefore possible to fall back on the already calculated results in the cache. Finally, it is also possible, e.g. in an iterative deepening framework, to perform move ordering after a search pass through the cache. A possible implementation of the [`principal variation search`](https://www.chessprogramming.org/Principal_Variation_Search), for example, uses the cache to determine the principal variation of the previous search pass.

First, we have to consider how an entry in the cache is structured. Besides the value of a node, the depth for which this value is valid is of course important. Furthermore, it could be seen with the `AlphaBeta algorithm` that it performs optimizations based on the `alpha` and `beta` value and thus on previously considered other paths in the tree. Thus, the value is not necessarily valid in a transposition that was reached by other means. One possibility would therefore be to store `alpha` and `beta` as well. 

A more meaningful way, however, is to classify the different nodes within a search tree:

1. `alpha < score < beta`: All child nodes have been examined and the value is always exact, even for transpositions. This is often referred to as `PV-Node`.
1. `alpha < beta <= score`: The search was aborted by a beta cut, therefore it is also called a `Cut-Node`. The score here is a lower limit, the actual value could be larger. 
1. `score <= alpha < beta`: Here all child nodes were examined too, however beta cutoffs occured. These nodes are also referred to as `All-Node`. Here the value acts as an upper bound, the actual value could be lower. 

In general it can be said that at least the root node and the leftmost node must be `PV-Nodes`, because `alpha` and `beta` cannot cause cuttofs there yet. The children of `PV-Nodes` can in turn also be `PV-Nodes` or `Cut-Nodes` if a beta cuttoff has occurred. After `Cut-Nodes` follow in each case alternating `All Nodes` and again `Cut-Nodes`. cite:[Node_Types].

The node type is stored in an enum. As designation the effect of the value was chosen, therefore, whether this is exact, a lower limit or an upper limit.

In [ ]:
from enum import Enum
from functools import total_ordering

# TODO: Explain Ordering


@total_ordering
class NodeType(Enum):
    EXACT = 0  # PV Node
    UPPER_BOUND = 1  # All Node
    LOWER_BOUND = 2  # Cut Node

    def __lt__(self, other):
        if self.__class__ is other.__class__:
            return self.value < other.value
        return NotImplemented

Based on this a dedicated data class is created
to store the node type, value and depth of a position.

In [ ]:
from dataclasses import dataclass
from chess.engine import PovScore


@dataclass
class CachedPositionEntry:
    type: NodeType
    value: PovScore
    depth: int

A dedicated class `AlphaBetaCache` is created
to provide an abstraction for storing and loading entries.
The entries are interally stored in a `dict` 
and wrapped methods are provided to reset the cache
or get its size.

In [ ]:
class AlphaBetaCache:

    def __init__(self):
        self.cache = dict()

    def clear(self):
        self.cache.clear()

    def size(self):
        return len(self.cache)

For a cache entry their needs to be a key to identify the position.
This key is calculated using the method `get_key`,
which takes a board as input and returns a key representing the position in a unique way.
As this function is called wether there is a matching cached entry or not
it needs to be fast.
A popular way of identifying positions in chess engines is the usage of [Zobrist Hashes](https://www.chessprogramming.org/Zobrist_Hashing),
as they can be determined incrementally cite:[Zobrist_Hashing].
As the `python-chess` library already has an internal method `_transposition_key`
this is used.
It is rather quick
as only internal representations of the position are used to calculate it
and the calculation is already done on every `push` and `pop`.
To use Zobrist Hashes in a useful way
the `python-chess` library would need to be patched.

In [ ]:
def get_key(self, board: chess.Board):
    return board._transposition_key()


AlphaBetaCache.get_key = get_key

To save an entry to the cache its node type needs to be calculated.
This is done by the auxilary function `_get_node_type`
which takes a value, the depth, alpha and beta as parameters and returns the node type.
It implements the typing as described above 
and additionally recognizes the leaf nodes as PV Nodes by their depth of 0.

In [ ]:
from chess.engine import Score


def _get_node_type(
    self, value: Score, depth: int, alpha: Score, beta: Score
) -> NodeType:
    if value <= alpha:
        return NodeType.UPPER_BOUND
    if value >= beta:
        return NodeType.LOWER_BOUND
    return NodeType.EXACT


AlphaBetaCache._get_node_type = _get_node_type

The `store_cache` method
adds an entry 
defined by the key returned by the `get_key` method
to the cache.
The entry consists of the node type as returned by the `_get_node_type` method,
the value and the current depth.
The parameters are therefore the parameters of `_get_node_type` (value, alpha, beta) and the key.

Initially the cache is checked for an already existing entry.
If there is a matching entry and its depth is deeper than the current depth, 
the current value is not stored in the cache 
as the other value has been calculated using more ressources already.
This is called depth first replacement
and is a popular way to handle replacing cache values,
but it is not the only way to do so. 
If there is no such entry,
the node type is evaluated and the tuple as described above is added to the cache.

In [ ]:
def store_cache(
    self, key: int, value: PovScore, depth: int, alpha: Score, beta: Score
):
    # depth first replacement
    entry = self.cache.get(key)
    if entry and entry.depth >= depth:
        return

    node_type = self._get_node_type(value.relative, depth, alpha, beta)
    self.cache[key] = CachedPositionEntry(node_type, value, depth)


AlphaBetaCache.store_cache = store_cache

The `load_cache` method returns an entry and it's type from the cache given a key.
Additionaly the depth is supplied as a parameter
as the cache entry is only relevant 
if it has the same or a greater depth.
If there is no matching entry
or the depth of the cached value is too shallow
a `KeyError` exception is thrown.

In [ ]:
from typing import Tuple


def load_cache(self, key: int, depth: int) -> Tuple[NodeType, PovScore]:
    entry = self.cache[key]
    if entry.depth < depth:
        raise KeyError

    return (entry.type, entry.value)


AlphaBetaCache.load_cache = load_cache

The class `AlphaBetaCache` is fully implemented,
but to use it the `_value` needs to be aware of it.
Therefore a decorator `cache_alpha_beta` is created
to check the cache for a matching entry
and to store the new result
if there was no matching entry yet.

The decorator calculates the key for the current position 
and tries to get a matching entry from the cache 
via `load_cache`.
If there is such an entry a differentiation needs to be made;
an exact value can be returned immediately
while a lower bound may increase `alpha`
and an upper bound may decrease `beta`.
This might lead to a cutoff
in case the new `alpha` is at least `beta`
and the cached value being returned immediately.

If the value is not returned immediately
or there was no value in the cache at all
the `value_function` is called
and the returned value is stored in the cache via `store_cache`
before returning the result.

In [ ]:
from chess.engine import Score, PovScore
from functools import wraps


def cache_alpha_beta(value_function):

    @wraps(value_function)
    def cached_value(
        self, board: chess.Board, depth: int, alpha: Score, beta: Score
    ) -> PovScore:
        cacheKey = self.cache.get_key(board)
        try:
            type, value = self.cache.load_cache(cacheKey, depth)

            if type == NodeType.EXACT:
                return value
            elif type == NodeType.LOWER_BOUND:
                alpha = max(value.relative, alpha)
            else:  # type == NodeType.UPPER_BOUND
                beta = min(value.relative, beta)

            if alpha >= beta:
                return value
        except:
            pass

        value = value_function(self, board, depth, alpha, beta)
        self.cache.store_cache(cacheKey, value, depth, alpha, beta)
        return value

    return cached_value

To use the cache a dedicated class `IterativeAlphaBetaCached` is created inheriting from `IterativeAlphaBeta`.
On construction of an object of the `IterativeAlphaBetaCached` class
the cache is created
and each call of `_evaluate_moves` empties the cache
to prevent stale data influencing the analysis.
To really use the cache
the `_value` function gets the newly created `cache_alpha_beta` decorator.

In [ ]:
from chess.engine import Score, PovScore


class IterativeAlphaBetaCached(IterativeAlphaBeta):

    def __init__(self, evaluator: Evaluator, max_look_ahead_depth: int):
        super().__init__(evaluator, max_look_ahead_depth)
        self.cache = AlphaBetaCache()

    def _evaluate_moves(self, board: chess.Board):
        self.cache.clear()
        return super()._evaluate_moves(board)

    @cache_alpha_beta
    def _value(
        self, board: chess.Board, depth: int, alpha: Score, beta: Score
    ) -> PovScore:
        return super()._value(board, depth, alpha, beta)

As the cache usage should only influence the performance
the result of the caching engine should match that of the non-caching engine.
This is checked here as seen in previous examples.

In [ ]:
import utils.min_max_tree
import random

random.seed(42)

engine = IterativeAlphaBetaCached(
    evaluator=standard_evaluator, max_look_ahead_depth=3
)
tree = utils.min_max_tree.add_tree_to_engine(engine)
result_iterativeAlphaBetaCached = engine.analyse(middlegame_board)

assert result_iterativeAlphaBetaCached == result_iterativeAlphaBeta